## Query consDB

Craig Lage - 17-Apr-25

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
from lsst.summit.utils import ConsDbClient

In [ ]:
client = ConsDbClient('http://consdb-pq.consdb:8080/consdb')
print(client.schema())  # list the instruments
print(client.schema('latiss'))  # list tables for an instrument

In [ ]:
# Get all of the values for a particular table and visit
query = 'SELECT * from cdb_latiss.exposure '
#query += 'cdb_latiss.exposure '
query += 'where day_obs = 20251112'
data = client.query(query)
for name in data.colnames:
    print(name, sys.getsizeof(data[name]))

In [ ]:
data['pgs_region']

In [ ]:
# Get all of the values for a particular table and visit
data = client.query('SELECT * from cdb_lsstcam.exposure_quicklook')
data.sort('exposure_id')
for i in range(len(data)):
    if data['mount_motion_image_degradation'][i] > 0.25:
        print(f"{data['exposure_id'][i]}  {data['mount_motion_image_degradation'][i]}")


In [ ]:
# Get all of the values for a particular table and visit
data = client.query('SELECT exposure_id, mount_motion_image_degradation from cdb_lsstcam.exposure_quicklook')
data.sort('exposure_id')
for i in range(len(data)):
    if data['mount_motion_image_degradation'][i] > 0.25:
        print(f"{data['exposure_id'][i]}  {data['mount_motion_image_degradation'][i]}")


In [ ]:
client.schema('lsstcam', 'exposure_quicklook') # List values in a table

In [ ]:
# Get all of the values for a particular table and visit
data = client.query('SELECT * from cdb_lsstcam.exposure_quicklook')
print(len(data))
for name in data.colnames:
    print(name, data[name].value[0])

In [ ]:
day_obs = 20251128
seq_min = 0
seq_max = 900
instrument = 'lsstcam'

visits_query = f'''
SELECT 
ccdvisit1_quicklook.psf_sigma,
ccdvisit1.detector,
visit1.visit_id,
visit1.seq_num,
visit1.day_obs,
visit1.airmass
FROM
cdb_{instrument}.ccdvisit1_quicklook AS ccdvisit1_quicklook,
cdb_{instrument}.ccdvisit1 AS ccdvisit1,
cdb_{instrument}.visit1_quicklook AS visit1_quicklook,
cdb_{instrument}.visit1 AS visit1 
WHERE 
ccdvisit1.ccdvisit_id = ccdvisit1_quicklook.ccdvisit_id
AND ccdvisit1.visit_id = visit1.visit_id 
AND visit1.visit_id = visit1_quicklook.visit_id
AND ccdvisit1.detector NOT IN (168, 188, 123, 27, 0, 20, 65, 161)
AND visit1.day_obs = {day_obs}
AND (visit1.seq_num BETWEEN {seq_min} AND {seq_max})
'''

ccdvisits = client.query(visits_query).to_pandas()
print(len(ccdvisits))
print(max(ccdvisits['seq_num'].values))

In [ ]:
day_obs = 20251128
seq_min = 0
seq_max = 900
instrument = 'lsstcam'

visits_query = f'''
SELECT 
ccdvisit1.detector,
visit1.visit_id,
visit1.seq_num,
visit1.day_obs,
visit1.airmass
FROM
cdb_{instrument}.ccdvisit1 AS ccdvisit1,
cdb_{instrument}.visit1 AS visit1 
WHERE 
ccdvisit1.visit_id = visit1.visit_id 
AND ccdvisit1.detector NOT IN (168, 188, 123, 27, 0, 20, 65, 161)
AND visit1.day_obs = {day_obs}
AND (visit1.seq_num BETWEEN {seq_min} AND {seq_max})
'''

ccdvisits = client.query(visits_query).to_pandas()
print(len(ccdvisits))
print(max(ccdvisits['seq_num'].values))

In [ ]:
day_obs = 20251112
query = f'''
SELECT  
exposure.azimuth,
exposure.wind_dir,
exposure.wind_speed,
exposure_quicklook.mount_motion_image_degradation  
FROM 
cdb_latiss.exposure,
cdb_latiss.exposure_quicklook 
WHERE 
cdb_latiss.exposure.day_obs = {day_obs} 
AND
cdb_latiss.exposure_quicklook.day_obs = {day_obs} 
AND 
cdb_latiss.exposure.exposure_id = cdb_latiss.exposure_quicklook.exposure_id
'''
#print(query)
data = client.query(query).to_pandas()
len(data)

In [ ]:

fig, axes = plt.subplots(2,1, figsize=(10, 10))
plt.subplots_adjust(hspace=0.3)
plt.suptitle(f"Mount motion fails vs azimuth - {day_obs}")
axes[0].scatter(data['azimuth'].values, data['mount_motion_image_degradation'].values)
axes[0].axvline(np.nanmedian(data['wind_dir']), ls = '--', color='k', label = "Mean wind direction")
axes[0].legend()
axes[0].set_xlabel("Azimuth (degrees)")
axes[0].set_ylabel("Mount motion image degradation (arcseconds)")

axes[1].scatter(data['azimuth'].values, data['wind_speed'].values)
axes[1].set_xlabel("Azimuth (degrees)")
axes[1].set_ylabel("Wind speed (m/sec)")

plt.savefig(f"/home/cslage/DATA/Motion_fails_vs_azimuth_{day_obs}.png")

In [ ]:
data['wind_dir']

In [ ]:
plt.plot(data['seq_num'].values)